In [1]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta,time
from streamlit_gsheets import GSheetsConnection
from auxiliar.auxiliar import *
from auxiliar.omie_aux import *

# testar subir OS

In [ ]:
base_compilada = pd.read_csv("testes\dados_testes.csv")
print(f"Tamanho da base: {base_compilada.shape}")


In [ ]:
clientes_omie = atualizar_base_clientes()
st.write(f"{len(clientes_omie)} clientes novos criados")

In [ ]:
base_compilada.columns

In [ ]:
id_cliente = 1037216

dados_cliente_str = (
    base_compilada.loc[base_compilada["customer_id"] == id_cliente, "dados_cliente"]
    .iat[0]                 # retorna o escalar, não o repr do Series/DataFrame
)

dados_cliente = json.loads(dados_cliente_str)
print(dados_cliente)

In [3]:
base_compilada = pd.read_csv("testes\dados_testes.csv")

clientes_subidos = criar_clientes_selecionados(base_compilada)
clientes_subidos = clientes_subidos.to_dict(orient='records')
st.write(f"{len(clientes_subidos)} clientes novos criados")
subir_dados_mongodb("log_clientes",clientes_subidos)

# st.write("Criando Ordens de Serviço...")
# os_subidos = criar_ordens_de_servico_da_planilha(base_compilada)
# os_subidos_dic = os_subidos.to_dict(orient='records')
# subir_dados_mongodb("log_os",os_subidos_dic)

# erros_mask = os_subidos["resposta"].astype(str).str.contains("ERROR", case=False, na=False)
# ids_para_subir = os_subidos.loc[~erros_mask,"os_id"]
# base_para_subir = dados_crm_df.loc[dados_crm_df['os_id'].isin(ids_para_subir)]
# base_para_subir = base_para_subir.drop_duplicates()
# base_para_subir_dic = base_para_subir.to_dict(orient='records')
# subir_dados_mongodb("os_processados",base_para_subir_dic)

2025-05-14 11:58:55.561 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 11:58:55.589 "runner.magic_enabled" is not a valid config option. If you previously had this config option set, it may have been removed.
2025-05-14 11:58:55.591 
  command:

    streamlit run c:\Users\ciros\github\gerador_de_notas_procorpo\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-14 11:58:55.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 11:58:55.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 11:58:55.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-14 11:58:55.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare m

id_cliente: 1037216 - Resultado: Error - Resposta: ERROR: O endereço de e-mail não é válido.Certifique-se de que possui uma conta e um domínio ( exemplo: minhaconta@dominio.com ) - tag [email] 
id_cliente: 812518 - Resultado: Error - Resposta: ERROR: API bloqueada por consumo indevido. Tente novamente em 1799 segundos.
id_cliente: 570203 - Resultado: Error - Resposta: ERROR: API bloqueada por consumo indevido. Tente novamente em 1798 segundos.
id_cliente: 718703 - Resultado: Error - Resposta: ERROR: API bloqueada por consumo indevido. Tente novamente em 1797 segundos.
ERROR: API bloqueada por consumo indevido. Tente novamente em 1796 segundos.
Muitos erros, parando a execução.
['ERROR: O endereço de e-mail não é válido.Certifique-se de que possui uma conta e um domínio ( exemplo: minhaconta@dominio.com ) - tag [email] ', 'ERROR: API bloqueada por consumo indevido. Tente novamente em 1799 segundos.', 'ERROR: API bloqueada por consumo indevido. Tente novamente em 1798 segundos.', 'ERROR:

AttributeError: 'NoneType' object has no attribute 'to_dict'

In [ ]:
linha_teste = base_compilada.iloc[0]
subir_linha_teste(linha_teste)

In [ ]:
codigo_integracao = pegar_dados_mongodb("id_clientes")
codigo_integracao

In [ ]:
id_do_cliente = 995654
unidade = "BackOffice"

mesmo_id  = codigo_integracao["codigo_cliente_integracao"] == id_do_cliente
mesma_unidade = codigo_integracao["unidade"] == unidade
if (mesmo_id & mesma_unidade).any():
    print("Cliente já existe na base")
# codigo_integracao.loc[codigo_integracao['codigo_cliente_integracao'] == 995654]
